<a href="https://colab.research.google.com/github/starkjiang/TrAC-GNN/blob/main/gae_vgae_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAE - VGAE Tutorial

In this tutorial, we will learn how to implement graph Autoencoder (GAE) and variational graph Autoencoder (VGAE) for the link prediction in a graph.

# 1. Installation

In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.5 MB/s eta 0:00:00


In [ ]:
import torch

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 2. Implementation

We still use CORA dataset in this tutorial.

In [ ]:
import argparse
import os.path as osp
import time

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GAE, VGAE, GCNConv

parser = argparse.ArgumentParser()
parser.add_argument('--variational', action='store_true')
parser.add_argument('--linear', action='store_true')
parser.add_argument('--dataset', type=str, default='Cora',
                    choices=['Cora', 'CiteSeer', 'PubMed'])
parser.add_argument('--epochs', type=int, default=400)
args = parser.parse_args(args=[])

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      split_labels=True, add_negative_train_samples=False),
])
dataset = Planetoid('.', name='Cora', transform=transform)
train_data, val_data, test_data = dataset[0]


# Define the encoder for GAE with GCN layer.
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv2 = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

# Define the encoder for VGAE with GCN layer.
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

# Define the encoder for GAE with linear layer.
class LinearEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = GCNConv(in_channels, out_channels)

    def forward(self, x, edge_index):
        return self.conv(x, edge_index)

# Define the encoder for VGAE with linear layer.
class VariationalLinearEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_mu = GCNConv(in_channels, out_channels)
        self.conv_logstd = GCNConv(in_channels, out_channels)

    def forward(self, x, edge_index):
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)


in_channels, out_channels = dataset.num_features, 16

if not args.variational and not args.linear:
    model = GAE(GCNEncoder(in_channels, out_channels))
elif not args.variational and args.linear:
    model = GAE(LinearEncoder(in_channels, out_channels))
elif args.variational and not args.linear:
    model = VGAE(VariationalGCNEncoder(in_channels, out_channels))
elif args.variational and args.linear:
    model = VGAE(VariationalLinearEncoder(in_channels, out_channels))

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    loss = model.recon_loss(z, train_data.pos_edge_label_index)
    if args.variational:
        loss = loss + (1 / train_data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    return model.test(z, data.pos_edge_label_index, data.neg_edge_label_index)


times = []
for epoch in range(1, args.epochs + 1):
    start = time.time()
    loss = train()
    auc, ap = test(test_data)
    print(f'Epoch: {epoch:03d}, AUC: {auc:.4f}, AP: {ap:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

Processing...
Done!


Epoch: 001, AUC: 0.6868, AP: 0.7155
Epoch: 002, AUC: 0.6831, AP: 0.7128
Epoch: 003, AUC: 0.6816, AP: 0.7122
Epoch: 004, AUC: 0.6812, AP: 0.7128
Epoch: 005, AUC: 0.6826, AP: 0.7146
Epoch: 006, AUC: 0.6839, AP: 0.7175
Epoch: 007, AUC: 0.6843, AP: 0.7195
Epoch: 008, AUC: 0.6839, AP: 0.7206
Epoch: 009, AUC: 0.6836, AP: 0.7210
Epoch: 010, AUC: 0.6826, AP: 0.7205
Epoch: 011, AUC: 0.6816, AP: 0.7197
Epoch: 012, AUC: 0.6810, AP: 0.7202
Epoch: 013, AUC: 0.6809, AP: 0.7208
Epoch: 014, AUC: 0.6829, AP: 0.7228
Epoch: 015, AUC: 0.6872, AP: 0.7244
Epoch: 016, AUC: 0.6931, AP: 0.7268
Epoch: 017, AUC: 0.6948, AP: 0.7264
Epoch: 018, AUC: 0.6945, AP: 0.7258
Epoch: 019, AUC: 0.6943, AP: 0.7250
Epoch: 020, AUC: 0.6985, AP: 0.7264
Epoch: 021, AUC: 0.7135, AP: 0.7331
Epoch: 022, AUC: 0.7384, AP: 0.7474
Epoch: 023, AUC: 0.7591, AP: 0.7624
Epoch: 024, AUC: 0.7702, AP: 0.7713
Epoch: 025, AUC: 0.7747, AP: 0.7748
Epoch: 026, AUC: 0.7779, AP: 0.7771
Epoch: 027, AUC: 0.7832, AP: 0.7821
Epoch: 028, AUC: 0.7904, AP: